In [1]:
import pyspark.sql
from pyspark.sql import SparkSession
from super_secret_password import PASSWORD

# PYSPARK INITALIZE
session = SparkSession.builder.appName('CLI User Interface Test').getOrCreate()

# TODO make a password saver or something

DATABASE_VALUES= {"url": "jdbc:mysql://localhost:3306/creditcard_capstone",
                "table":["CDW_SAPP_BRANCH", "CDW_SAPP_CREDIT_CARD", "CDW_SAPP_CUSTOMER"],
                "mode":"overwrite",
                "properties":{"user":"root", "password":PASSWORD, "driver":"com.mysql.cj.jdbc.Driver"}}


In [119]:
import pyspark.sql.functions as F

In [27]:


query = """
    SELECT *
    FROM 
    WHERE CUST_ZIP = '17201' AND YEAR(TIMEID) = '2018' AND MONTH(TIMEID) = 'September'
    ORDER BY DAYOFMONTH(cdw_sapp_credit_card.TIMEID) DESC 
"""


# sql_db_df_custom = session.read.format("jdbc").options(
#     driver="com.mysql.cj.jdbc.Driver",
#     user="root",
#     password=PASSWORD,  
#     url=DATABASE_VALUES["url"],
#     dbtable=query).load()


In [23]:

sql_branch_df = session.read.jdbc(
        url=DATABASE_VALUES["url"],
        properties=DATABASE_VALUES["properties"],
        table=DATABASE_VALUES["table"][0]
    )

In [178]:
sql_credit_df = session.read.jdbc(
        url=DATABASE_VALUES["url"],
        properties=DATABASE_VALUES["properties"],
        table=DATABASE_VALUES["table"][1]
    )

In [20]:
sql_customer_df = session.read.jdbc(
        url=DATABASE_VALUES["url"],
        properties=DATABASE_VALUES["properties"],
        table=DATABASE_VALUES["table"][2]
    )

In [179]:
sql_credit_customer_df = sql_customer_df.join(sql_credit_df, sql_customer_df.SSN == sql_credit_df.CUST_SSN)

In [180]:
sql_credit_customer_df = sql_credit_customer_df.drop('CUST_SSN')

In [181]:
sql_credit_customer_df = sql_credit_customer_df.drop(sql_customer_df.CREDIT_CARD_NO)

In [182]:
sql_credit_customer_df.printSchema()

root
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: long (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- TIMEID: date (nullable = true)



In [183]:
sql_credit_customer_df.createOrReplaceTempView("credit_customer_df")

In [214]:
transaction_container_query = {"zipcode":"",
                                "type":"",
                                "state":"",
                                "branches":"",
                                "year":"2018",
                                "month":""}

In [215]:
transaction_container_query["zipcode"] = '17201'


In [223]:
transaction_container_query['month'] = 'september'

In [224]:
# zipcode 
# month
# year

#  dates
# date_format(date, selection)
# DAYOFMONTH(TIMEID)

transaction_menu_choice_one = session.sql(f"""
    SELECT 
        CUST_ZIP,
        FIRST_NAME,
        LAST_NAME,
        CREDIT_CARD_NO,
        TRANSACTION_ID, 
        TRANSACTION_TYPE, 
        TRANSACTION_VALUE, 
        TIMEID,

        DATE_FORMAT(TIMEID, 'MMMM')
    FROM credit_customer_df
    WHERE 
        CUST_ZIP = {transaction_container_query["zipcode"]}
        AND 
        YEAR(TIMEID) = {transaction_container_query["year"]}
        AND 
        DATE_FORMAT(TIMEID, 'MMMM') LIKE("{transaction_container_query['month']}%")
        

    ORDER BY DAYOFMONTH(TIMEID) DESC; """)

In [225]:
transaction_menu_choice_one.show(10)

+--------+----------+---------+----------------+--------------+----------------+-----------------+----------+------------------+-------------------------+
|CUST_ZIP|FIRST_NAME|LAST_NAME|  CREDIT_CARD_NO|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|    TIMEID|dayofmonth(TIMEID)|date_format(TIMEID, MMMM)|
+--------+----------+---------+----------------+--------------+----------------+-----------------+----------+------------------+-------------------------+
|   17201|      Otha| Atkinson|4210653324260789|         17117|           Bills|             1.02|2018-09-28|                28|                September|
|   17201|    Brooke|    Neely|4210653373620931|         14169|             Gas|            73.58|2018-09-28|                28|                September|
|   17201|    Marcel|     Camp|4210653344660822|         46659|   Entertainment|            47.92|2018-09-27|                27|                September|
|   17201|    Rickey|  Robbins|4210653374041659|         13490|       